```
From: https://github.com/ksatola
Version: 0.0.0
```

# Missing Data

We need to deal with missing data as most algorithms will not work if data is missing. Notable exceptions are the recent boosting libraries: XGBoost, CatBoost, and LightGBM. There are no hard answers for how to treat missing data. Also, missing data could represent different situations.

There are various ways to handle missing data:
- Remove any row with missing data,
- Remove any column with missing data,
- Impute missing values,
- Create an indicator column to signify data was missing.



In [1]:
# Connect with underlying Python code
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, 'src')

In [2]:
from datasets import (
    get_dataset
)

In [3]:
df = get_dataset('titanic3')